# Get Captions

In [1]:
playlist_input = "https://www.youtube.com/playlist?list=PLmbUlSQHQlvhEqMXztLb2ZnLDvg20J2nD"

In [2]:
import re

playlist_id = None
# Regex to extract playlist ID from various YouTube URL formats
playlist_url_regex = r'(?:youtube\.com\/(?:[^\/]+\/.+?\/|(?:v|e(?:mbed)?)\/|.*[?&]list=)|youtu\.be\/)([^"&?\/ ]{11,})'

match = re.search(playlist_url_regex, playlist_input)
if match:
    playlist_id = match.group(1)

# If it's not a URL or the regex failed, assume the input is directly the playlist ID
if not playlist_id:
    playlist_id = playlist_input

print(f"Extracted Playlist ID: {playlist_id}")

Extracted Playlist ID: PLmbUlSQHQlvhEqMXztLb2ZnLDvg20J2nD


In [30]:
import sys
!{sys.executable} -m pip install --upgrade google-api-python-client
print("Installed google-api-python-client.")

Installed google-api-python-client.


In [3]:
from googleapiclient.discovery import build
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("YOUTUBE_API_KEY")
# Placeholder for your API key. Replace with your actual key.
YOUTUBE_API_KEY = api_key

youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)
print("YouTube API service initialized.")

YouTube API service initialized.


In [4]:
max_videos = 10000
print(f"Max videos to process set to: {max_videos}")

Max videos to process set to: 10000


In [5]:
video_ids = []
next_page_token = None

while True:
    request = youtube.playlistItems().list(
        part="snippet",
        playlistId=playlist_id,
        maxResults=50, # Max results per page
        pageToken=next_page_token
    )
    response = request.execute()

    for item in response['items']:
        video_id = item['snippet']['resourceId']['videoId']
        video_ids.append(video_id)
        if len(video_ids) >= max_videos:
            break

    next_page_token = response.get('nextPageToken')
    if not next_page_token or len(video_ids) >= max_videos:
        break

# Truncate video_ids to ensure it does not exceed max_videos
# video_ids = video_ids[:max_videos]

print(f"Found {len(video_ids)} video IDs.")
print("Video IDs:", video_ids)

Found 682 video IDs.
Video IDs: ['k5juTqk-OoA', 'VQo1Eut6jhA', 'hvHNbeTsneY', 'OgYJmEB7lQk', 'Zs7qVzVu8sA', '4raib3FSIdE', 'EHXRd4oNEqM', 'KA46LrBO7TM', '47KrW28W2YU', 'HEYfSKI-kWw', 'rK90XkzEvd0', 'dpwfv92XUIM', 'pbJIaUr-Sfg', 'c5C9wn6Kd-E', '1sS7XPc4PnQ', '14uO75YlYkA', 'k27pd5LLxEs', 'y3_4F0_PM_0', 'vaMPbvvFLXI', '5A2J7jSyTAo', 'mSD3QG-5Xzg', 'rnSRXbK7H4g', 'nBfKZpku4Vo', '5gcVK_XvxSQ', 'fngLDf_K0Lw', 'b5FAVx8h-MU', '3M0ziiToATw', 'Y0-NcZBwYxo', 'OlQ1ILcyXfk', 'fpQ1pmBX5Qk', 'Ed8NqOOzNkc', 'Wcrpxsc1sT0', 'CGC-y3ySlhA', 'DSeg7Hx_GLc', 'u5UbGUuvBnI', 'vtT-QtiuwxY', 'mla3BXFGXXk', 'JPpJhsUGn6w', 'JPKJiJJDBqU', 'fgF5u05wTYM', 'AlBJwRqfD8A', 'FsrIbZfjc38', 'dU4B3HrgBxM', 'TEJxyq-I2AM', 'MJbUW84XCho', '2jfISLM_ZDY', '7C2M9qDaMIU', '6lA_9iMf7Wk', '-YKYepbihEI', 'jkD0EzqUj4o', 'ciH6t0VMw-c', 'raFnAJ1g_tU', 'tTsR_1iMYXI', 's3RV5Yjze3Q', '7wS_pfMz90M', 'FTrMuF6A_bc', 'vSaYg7BF6Hg', 'bND2kvgqQ3E', 'XCidDCrvI1E', 'dInifOpWZqg', 'mi-RiCAEdRs', 'u4NwDAi5ab0', 'Ry_Fh9W7O6k', 'T-1eGfIE2FU', 'zaMR4v

In [4]:
video_ids = ['cEUjCBuNsoiz1RnM', 'PEoQbkybDoiMk50L','i3M3LxQ2I', 'zA8u0b3gqpc', 'aBt59UADTjY', '0ukTuuwsFzuzAqIi','HeHBa6adM','7dwjekfE1tfCtoCp', 'OSc_ZJWJsgMFTo5I']

In [1]:
video_urls= ['https://www.youtube.com/live/k5vOdgMdLyU?si=cEUjCBuNsoiz1RnM', 'https://www.youtube.com/live/K4IxPBLZKoI?si=PEoQbkybDoiMk50L', 'https://youtu.be/v-i3M3LxQ2I', 'https://youtu.be/zA8u0b3gqpc', 'https://youtu.be/aBt59UADTjY', 'https://www.youtube.com/live/_BpU0QV3W6o?si=0ukTuuwsFzuzAqIi', 'https://youtu.be/u-HeHBa6adM', 'https://www.youtube.com/live/TnR66cqSkbc?si=7dwjekfE1tfCtoCp', 'https://www.youtube.com/live/nnIts12H0Fo?si=OSc_ZJWJsgMFTo5I']

**Reasoning**:
The next step is to create a directory named `captions` to store the downloaded caption files.



In [6]:
import os

captions_dir = "captions"
os.makedirs(captions_dir, exist_ok=True)
print(f"Directory '{captions_dir}' created or already exists.")

Directory 'captions' created or already exists.


In [8]:
import yt_dlp
import os

for video_url in video_ids:
        ydl_opts = {
            'writesubtitles': True,
            'writeautomaticsub': True,
            'subtitleslangs': ['pt'],  
            'subtitlesformat': 'srt',
            'skip_download': True,
            'outtmpl': os.path.join(captions_dir, '%(title)s-%(id)s.%(ext)s'),
            'quiet': True,  # Set to True to suppress output
            'warnings': True
        }

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(video_url, download=False)
                # Now try to download
                ydl.download([video_url])
                print(f"✓ Successfully downloaded captions for: {video_url}")
                
        except Exception as e:
            print(f"✗ Could not download captions for {video_url}. Error: {e}")

print(f"\nCaption download completed for {len(video_url)} videos.")

✓ Successfully downloaded captions for: k5juTqk-OoA      


✓ Successfully downloaded captions for: VQo1Eut6jhA   


ERROR: [youtube] hvHNbeTsneY: Video unavailable. This video has been removed by the uploader


✗ Could not download captions for hvHNbeTsneY. Error: ERROR: [youtube] hvHNbeTsneY: Video unavailable. This video has been removed by the uploader


✓ Successfully downloaded captions for: OgYJmEB7lQk     


...

✗ Could not download captions for uAUCYx6DV3U. Error: ERROR: [youtube] uAUCYx6DV3U: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


ERROR: [youtube] vufs7fxkQ9M: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


✗ Could not download captions for vufs7fxkQ9M. Error: ERROR: [youtube] vufs7fxkQ9M: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


ERROR: [youtube] x4PALDeVwqM: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


✗ Could not download captions for x4PALDeVwqM. Error: ERROR: [youtube] x4PALDeVwqM: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies

Caption download completed for 11 videos.


# Proccess Captions

In [2]:
import os
import pysrt
from typing import Annotated, Literal, TypedDict
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

### --- PARTE 1: EXTRAÇÃO AUTOMÁTICA DE IDs DOS ARQUIVOS BAIXADOS ---

In [1]:
import os, re, pysrt
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_chroma import Chroma

load_dotenv()

def extrair_id(nome):
    m = re.search(r'-([a-zA-Z0-9_-]{11})\.(?:[a-z]{2}\.)?srt$', nome)
    return m.group(1) if m else None


DB_PATH = "./db_clone"
CAPTIONS_DIR =  "./captions"  # ajuste se necessário

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    encode_kwargs={"normalize_embeddings": True},
)
vectordb = Chroma(persist_directory=str(DB_PATH), embedding_function=embeddings, collection_metadata={"hnsw:space": "cosine"})

docs = []
for fn in os.listdir(CAPTIONS_DIR):
    if not fn.endswith(".srt"):
        continue
    vid = extrair_id(fn)
    if not vid:
        continue

    path = CAPTIONS_DIR + "/" +fn
    try:
        subs = pysrt.open(str(path), encoding="utf-8")
    except:
        subs = pysrt.open(str(path), encoding="iso-8859-1")


    CHUNK_SIZE = 1000 # caracteres por chunk
    OVERLAP = 50      # sobreposição

    buffer = []
    buffer_text = ""

    for sub in subs:
        buffer.append(sub)
        buffer_text += " " + sub.text
        
        if len(buffer_text) >= CHUNK_SIZE:
            text = " ".join(s.text for s in buffer)
            text = re.sub(r"\s+", " ", text).strip()
            
            if len(text) >= 100:
                start = buffer[0].start
                sec = start.hours*3600 + start.minutes*60 + start.seconds
                url = f"https://youtu.be/{vid}?t={sec}s"
                
                docs.append(Document(
                    page_content=text,
                    metadata={"url": url, "fonte": fn, "video_id": vid, "t": sec}
                ))
            
            # Mantém overlap
            buffer = buffer[-3:]
            buffer_text = " ".join(s.text for s in buffer)

# Processa resto
if buffer and len(buffer_text) >= 100:
    text = " ".join(s.text for s in buffer)
    start = buffer[0].start
    sec = start.hours*3600 + start.minutes*60 + start.seconds
    url = f"https://youtu.be/{vid}?t={sec}s"
    docs.append(Document(
        page_content=text,
        metadata={"url": url, "fonte": fn, "video_id": vid, "t": sec}
    ))

if not docs:
    raise SystemExit("Nenhum documento gerado. Verifique CAPTIONS_DIR e nomes dos .srt")

BATCH = 2000  # must be < 5461 (your max); 1000–2000 is safe

for i in range(0, len(docs), BATCH):
    vectordb.add_documents(docs[i:i+BATCH])
    print(f"upserted {min(i+BATCH, len(docs))}/{len(docs)}")

try:
    vectordb.persist()
except Exception:
    pass
print("Inseridos:", len(docs))

# Some versions persist automatically; this makes it explicit if available
try:
    vectordb.persist()
except Exception:
    pass

print("DB_PATH:", DB_PATH)


C:\Temp\ipykernel_28432\541166905.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Programming\envs\CloneRenan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


upserted 2000/27441
upserted 4000/27441
upserted 6000/27441
upserted 8000/27441
upserted 10000/27441
upserted 12000/27441
upserted 14000/27441
upserted 16000/27441
upserted 18000/27441
upserted 20000/27441
upserted 22000/27441
upserted 24000/27441
upserted 26000/27441
upserted 27441/27441
Inseridos: 27441
DB_PATH: ./db_clone


In [9]:
load_dotenv()

# --- CONFIGURAÇÃO DO KNOWLEDGE BASE ---
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_db = Chroma(persist_directory="./db_clone", embedding_function=embeddings)
retriever = vector_db.as_retriever(search_kwargs={"k": 5})

# --- DEFINIÇÃO DA FERRAMENTA (TOOL) ---
@tool
def pesquisar_memoria_renan(query: str) -> str:
    """Busca trechos de colunas, transcrições e pensamentos do Renan Santos sobre um tema."""
    docs = retriever.invoke(query)
    # Formata como o Master Prompt espera (como memórias)
    contexto = "\n\n".join([f"[Trecho]: {d.page_content}" for d in docs])
    return contexto

# --- CONFIGURAÇÃO DO MODELO COM MASTER PROMPT ---
with open("prompt_clone.txt", "r", encoding="utf-8") as f:
    master_prompt = f.read()

# Substitua a criação do llm por:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    model_kwargs={"system_instruction": master_prompt} # Passando via kwargs
).bind_tools([pesquisar_memoria_renan])

# --- LÓGICA DO GRAFO (AGENT NODES) ---

def chatbot_renan(state: MessagesState):
    # O Gemini precisa do histórico completo para decidir se a ferramenta já foi usada
    # Injetamos o Master Prompt como SystemMessage aqui para garantir a identidade
    messages = [SystemMessage(content=master_prompt)] + state["messages"]
    
    # Chamada ao modelo
    response = llm.invoke(messages)
    
    # IMPORTANTE: Se o modelo decidir usar uma ferramenta, 
    # a resposta virá com 'tool_calls'. O LangGraph precisa disso.
    return {"messages": [response]}
# --- MONTAGEM DO FLUXO (LANGGRAPH) ---
workflow = StateGraph(MessagesState)

# 1. Adicionar os Nós
workflow.add_node("chatbot", chatbot_renan)
workflow.add_node("tools", ToolNode([pesquisar_memoria_renan]))

# 2. Definir as conexões (Arestas)
workflow.add_edge(START, "chatbot")

# O 'tools_condition' decide: se o LLM pediu tool -> vai para 'tools', senão -> termina
workflow.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# Após usar a ferramenta, ele volta para o chatbot para processar a informação
workflow.add_edge("tools", "chatbot")

# Compilar o Agente
app = workflow.compile()

NameError: name 'GoogleGenerativeAIEmbeddings' is not defined

In [11]:
os.getcwd()

'c:\\Users\\Luiz Venosa\\Documents\\Questions\\CloneRenan'

In [18]:
from pydub import AudioSegment
import os

def converter_m4a_para_wav(input_path, output_path="renan_sample.wav"):
    print(f"🔄 A converter {input_path} para formato compatível...")
    
    # Carrega o ficheiro m4a
    audio = AudioSegment.from_file(input_path, format="m4a")
    
    # Otimiza para o XTTSv2:
    # - Mono (1 canal)
    # - 22050Hz ou 16000Hz (frequências ideais para clonagem)
    audio = audio.set_channels(1).set_frame_rate(22050)
    
    # Exporta como WAV
    audio.export(output_path, format="wav")
    print(f"✅ Concluído! Ficheiro pronto: {output_path}")

if __name__ == "__main__":
    # Substitui pelo nome exato do teu ficheiro m4a
    nome_do_teu_ficheiro = "C:\\Users\\Luiz Venosa\\Documents\\Questions\\CloneRenan\\renan_sample.m4a" 
    if os.path.exists(nome_do_teu_ficheiro):
        converter_m4a_para_wav(nome_do_teu_ficheiro)
    else:
        print("❌ Ficheiro m4a não encontrado na pasta.")

🔄 A converter C:\Users\Luiz Venosa\Documents\Questions\CloneRenan\renan_sample.m4a para formato compatível...
✅ Concluído! Ficheiro pronto: renan_sample.wav
